# KisanMind Soil Classification Model Training

**Model**: DenseNet121 (transfer learning from ImageNet)

**Dataset**: Indian Region Soil Images

**Classes**: Clay, Sand, Loamy Sand, Sandy Loam, Loam

**Expected Training Time**: 1-2 hours on Kaggle GPU

**Target Accuracy**: 90-95%

---

## Setup Instructions

1. **Upload this notebook to Kaggle**
2. **Upload soil images**:
   - Create a Kaggle dataset with Sample1.jpg - Sample16.jpg
   - Include Practical_Reading.csv
3. **Enable GPU**: Settings → Accelerator → GPU T4 x2
4. **Run all cells**

---

In [ ]:
# Install dependencies
!pip install -q tensorflow==2.15.0
!pip install -q albumentations
!pip install -q tf2onnx
!pip install -q onnxruntime

print("✅ Dependencies installed")

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
import shutil
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Step 1: Data Preparation

Organize images into train/val/test directories by soil type

In [ ]:
# Read soil type labels
# UPDATE THIS PATH to point to your uploaded dataset
data_dir = '/kaggle/input/indian-soil-images'  # Change this!

df = pd.read_csv(f'{data_dir}/Practical_Reading.csv')
print("Dataset overview:")
print(df['Type'].value_counts())
print(f"\nTotal samples: {len(df)}")

In [ ]:
# Create organized directory structure
base_dir = '/kaggle/working/soil_data'
os.makedirs(base_dir, exist_ok=True)

for split in ['train', 'val', 'test']:
    for soil_type in df['Type'].unique():
        os.makedirs(f"{base_dir}/{split}/{soil_type}", exist_ok=True)

print("✅ Directory structure created")

In [ ]:
# Split data: 70% train, 15% val, 15% test
np.random.seed(42)

for soil_type in df['Type'].unique():
    samples = df[df['Type'] == soil_type]['Sample'].values
    
    # Split indices
    train_samples, temp_samples = train_test_split(
        samples, test_size=0.3, random_state=42
    )
    val_samples, test_samples = train_test_split(
        temp_samples, test_size=0.5, random_state=42
    )
    
    # Copy images to appropriate directories
    for sample_id in train_samples:
        src = f"{data_dir}/Sample{sample_id}.jpg"
        dst = f"{base_dir}/train/{soil_type}/Sample{sample_id}.jpg"
        if os.path.exists(src):
            shutil.copy(src, dst)
    
    for sample_id in val_samples:
        src = f"{data_dir}/Sample{sample_id}.jpg"
        dst = f"{base_dir}/val/{soil_type}/Sample{sample_id}.jpg"
        if os.path.exists(src):
            shutil.copy(src, dst)
    
    for sample_id in test_samples:
        src = f"{data_dir}/Sample{sample_id}.jpg"
        dst = f"{base_dir}/test/{soil_type}/Sample{sample_id}.jpg"
        if os.path.exists(src):
            shutil.copy(src, dst)
    
    print(f"{soil_type}: {len(train_samples)} train, {len(val_samples)} val, {len(test_samples)} test")

print("\n✅ Data split complete")

## Step 2: Data Augmentation

Heavy augmentation to compensate for small dataset (16 samples total)

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

# Validation data (no augmentation, just rescaling)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data
img_size = (224, 224)
batch_size = 8  # Small batch for small dataset

train_generator = train_datagen.flow_from_directory(
    f'{base_dir}/train',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    f'{base_dir}/val',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_datagen.flow_from_directory(
    f'{base_dir}/test',
    target_size=img_size,
    batch_size=1,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClass indices: {train_generator.class_indices}")
print(f"Number of classes: {len(train_generator.class_indices)}")

## Step 3: Build Model

DenseNet121 with transfer learning from ImageNet

In [ ]:
# Load pre-trained DenseNet121
base_model = DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model initially
base_model.trainable = False

# Build complete model
num_classes = len(train_generator.class_indices)

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])

print("Model architecture:")
model.summary()
print(f"\nTotal params: {model.count_params():,}")

## Step 4: Training Phase 1 - Frozen Base

Train only the top layers while keeping DenseNet frozen

In [ ]:
# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
)

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'soil_model_phase1.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("\n🚀 Starting Phase 1 training (frozen base)...\n")

# Train
history1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Will early stop
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Phase 1 training complete!")

## Step 5: Training Phase 2 - Fine-tuning

Unfreeze the base model and fine-tune with lower learning rate

In [ ]:
# Unfreeze base model
base_model.trainable = True

print(f"Number of trainable layers: {len([l for l in model.layers if l.trainable])}")

# Recompile with lower learning rate
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),  # Much lower LR
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.TopKCategoricalAccuracy(k=2, name='top_2_accuracy')]
)

# Update callbacks
callbacks_phase2 = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    tf.keras.callbacks.ModelCheckpoint(
        'soil_model_phase2.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=7,
        min_lr=1e-8,
        verbose=1
    )
]

print("\n🚀 Starting Phase 2 training (fine-tuning entire model)...\n")

# Fine-tune
history2 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,  # Will early stop
    callbacks=callbacks_phase2,
    verbose=1
)

print("\n✅ Phase 2 training complete!")

## Step 6: Evaluate Model

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Phase 1
axes[0, 0].plot(history1.history['accuracy'], label='Train')
axes[0, 0].plot(history1.history['val_accuracy'], label='Val')
axes[0, 0].set_title('Phase 1: Accuracy')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True)

axes[0, 1].plot(history1.history['loss'], label='Train')
axes[0, 1].plot(history1.history['val_loss'], label='Val')
axes[0, 1].set_title('Phase 1: Loss')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True)

# Phase 2
axes[1, 0].plot(history2.history['accuracy'], label='Train')
axes[1, 0].plot(history2.history['val_accuracy'], label='Val')
axes[1, 0].set_title('Phase 2: Accuracy (Fine-tuning)')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Accuracy')
axes[1, 0].legend()
axes[1, 0].grid(True)

axes[1, 1].plot(history2.history['loss'], label='Train')
axes[1, 1].plot(history2.history['val_loss'], label='Val')
axes[1, 1].set_title('Phase 2: Loss (Fine-tuning)')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Loss')
axes[1, 1].legend()
axes[1, 1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ Training plots saved")

In [ ]:
# Evaluate on test set
print("\n📊 Evaluating on test set...\n")

test_loss, test_acc, test_top2 = model.evaluate(test_generator, verbose=1)

print(f"\n🎯 Test Results:")
print(f"  Test Accuracy: {test_acc*100:.2f}%")
print(f"  Test Top-2 Accuracy: {test_top2*100:.2f}%")
print(f"  Test Loss: {test_loss:.4f}")

# Get predictions for confusion matrix
test_generator.reset()
predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

cm = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_labels, 
            yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Classification report
print("\n📋 Classification Report:\n")
print(classification_report(true_classes, predicted_classes, 
                          target_names=class_labels, digits=3))

## Step 7: Save Model

In [ ]:
# Save final Keras model
model.save('soil_classifier_densenet121_final.h5')
print("✅ Keras model saved: soil_classifier_densenet121_final.h5")

# Save model architecture and weights separately
model_json = model.to_json()
with open('soil_model_architecture.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('soil_model_weights.h5')

print("✅ Model architecture and weights saved separately")

# Save class mapping
import json
with open('soil_classes.json', 'w') as f:
    json.dump(train_generator.class_indices, f, indent=2)

print("✅ Class mapping saved: soil_classes.json")

## Step 8: Convert to ONNX

In [ ]:
import tf2onnx

print("🔄 Converting to ONNX format...")

# Convert to ONNX
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32, name="input"),)
output_path = "soil_classifier.onnx"

model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=spec,
    opset=13,
    output_path=output_path
)

print(f"✅ ONNX model saved: {output_path}")

# Check file size
import os
onnx_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"📦 ONNX model size: {onnx_size_mb:.2f} MB")

## Step 9: Quantize to INT8

In [ ]:
from onnxruntime.quantization import quantize_dynamic, QuantType

print("🔄 Quantizing to INT8...")

quantized_model_path = "soil_classifier_quantized.onnx"

quantize_dynamic(
    model_input=output_path,
    model_output=quantized_model_path,
    weight_type=QuantType.QInt8,
    optimize_model=True,
)

print(f"✅ Quantized model saved: {quantized_model_path}")

# Check quantized size
quantized_size_mb = os.path.getsize(quantized_model_path) / (1024 * 1024)
reduction = ((onnx_size_mb - quantized_size_mb) / onnx_size_mb) * 100

print(f"📦 Quantized model size: {quantized_size_mb:.2f} MB")
print(f"📉 Size reduction: {reduction:.1f}%")

## Step 10: Validate ONNX Model

In [ ]:
import onnxruntime as ort
import time

print("🧪 Validating ONNX model...\n")

# Load ONNX model
session = ort.InferenceSession(quantized_model_path)
input_name = session.get_inputs()[0].name

# Test on validation set
val_generator.reset()
onnx_predictions = []
inference_times = []

for i in range(len(val_generator)):
    batch = val_generator[i]
    images = batch[0]
    
    for img in images:
        img_input = np.expand_dims(img, axis=0)
        
        # Time inference
        start = time.time()
        output = session.run(None, {input_name: img_input})[0]
        inference_times.append((time.time() - start) * 1000)  # ms
        
        onnx_predictions.append(np.argmax(output))

# Calculate metrics
val_generator.reset()
true_labels = val_generator.classes
onnx_accuracy = np.mean(np.array(onnx_predictions) == true_labels)
avg_inference_time = np.mean(inference_times)

print(f"🎯 ONNX Validation Results:")
print(f"  Accuracy: {onnx_accuracy*100:.2f}%")
print(f"  Avg Inference Time: {avg_inference_time:.2f} ms")
print(f"  Min Inference Time: {np.min(inference_times):.2f} ms")
print(f"  Max Inference Time: {np.max(inference_times):.2f} ms")

# Acceptance criteria
print("\n" + "="*60)
print("🏆 ACCEPTANCE CRITERIA CHECK:")
print("="*60)

acc_pass = onnx_accuracy >= 0.85
time_pass = avg_inference_time < 200
size_pass = quantized_size_mb < 15

print(f"  Accuracy ≥85%: {'✅ PASS' if acc_pass else '❌ FAIL'} ({onnx_accuracy*100:.1f}%)")
print(f"  Inference <200ms: {'✅ PASS' if time_pass else '❌ FAIL'} ({avg_inference_time:.1f}ms)")
print(f"  Model size <15MB: {'✅ PASS' if size_pass else '❌ FAIL'} ({quantized_size_mb:.1f}MB)")

if acc_pass and time_pass and size_pass:
    print("\n🎉 ALL CRITERIA PASSED! Model ready for production deployment.")
else:
    print("\n⚠️  Some criteria not met. Consider retraining with more data or adjusting hyperparameters.")

## Step 11: Create Deployment Package

In [ ]:
import zipfile

print("📦 Creating deployment package...\n")

# Create deployment zip
with zipfile.ZipFile('kisanmind_soil_model_deployment.zip', 'w') as zipf:
    zipf.write(quantized_model_path)
    zipf.write('soil_classes.json')
    zipf.write('training_history.png')
    zipf.write('confusion_matrix.png')
    
    # Create README
    readme_content = f"""# KisanMind Soil Classification Model

## Model Details
- Architecture: DenseNet121 (transfer learning)
- Input: 224x224 RGB images
- Output: {num_classes} soil type classes
- Format: ONNX (INT8 quantized)

## Performance Metrics
- Test Accuracy: {test_acc*100:.2f}%
- ONNX Accuracy: {onnx_accuracy*100:.2f}%
- Inference Time: {avg_inference_time:.2f} ms (CPU)
- Model Size: {quantized_size_mb:.2f} MB

## Classes
{json.dumps(train_generator.class_indices, indent=2)}

## Usage
```python
import onnxruntime as ort
import numpy as np
from PIL import Image

# Load model
session = ort.InferenceSession('soil_classifier_quantized.onnx')

# Preprocess image
img = Image.open('soil.jpg').resize((224, 224))
img_array = np.array(img).astype(np.float32) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
input_name = session.get_inputs()[0].name
output = session.run(None, {{input_name: img_array}})[0]
predicted_class = np.argmax(output)
confidence = output[0][predicted_class]

print(f"Predicted: Class {{predicted_class}}, Confidence: {{confidence:.2%}}")
```

## Training Date
{pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

## Dataset
Indian Region Soil Image Dataset (Kaggle)
- Total samples: {len(df)}
- Training: {len(train_generator.filenames)}
- Validation: {len(val_generator.filenames)}
- Test: {len(test_generator.filenames)}
"""
    
    zipf.writestr('README.md', readme_content)

print("✅ Deployment package created: kisanmind_soil_model_deployment.zip")
print("\n📥 Download this file and deploy to KisanMind ML service!")

# Show contents
print("\n📋 Package contents:")
with zipfile.ZipFile('kisanmind_soil_model_deployment.zip', 'r') as zipf:
    for info in zipf.infolist():
        print(f"  - {info.filename} ({info.file_size / 1024:.1f} KB)")

## ✅ Training Complete!

### Next Steps:

1. **Download deployment package**: `kisanmind_soil_model_deployment.zip`
2. **Extract files to**: `services/ml-inference/models/`
3. **Update API code** to use the new ONNX model
4. **Test locally** before deploying to Render
5. **Deploy to production** on Render

### Model Performance Summary:

| Metric | Value | Target | Status |
|--------|-------|--------|--------|
| Test Accuracy | {:.1%} | ≥85% | {} |
| Inference Time | {:.1f}ms | <200ms | {} |
| Model Size | {:.1f}MB | <15MB | {} |

### Training Notes:

⚠️ **Small Dataset Warning**: This model was trained on only 16 samples. For production use:
- Collect more real-world soil images (target: 500+ per class)
- Use PlantDoc-style field images for better generalization
- Implement active learning to improve with user feedback
- Consider ensemble with other soil models

---

**Generated by**: KisanMind ML Training Pipeline

**Training Platform**: Kaggle (Free GPU)

**Model Version**: 1.0.0".format(
    test_acc,
    '✅' if acc_pass else '❌',
    avg_inference_time,
    '✅' if time_pass else '❌',
    quantized_size_mb,
    '✅' if size_pass else '❌'
)